### Import the necessary libraries

In [114]:
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

### Import the processed data and create the features and target arrays

In [2]:
abalone_df = pd.read_csv("abalone_processed.csv")
abalone_df

,Length,Diam,Height,Whole,Shucked,Viscera,Shell,Sex_F,Sex_I,Sex_M,Rings
0,-0.592283,-0.433414,-1.199002,-0.625502,-0.604416,-0.719291,-0.619496,0.0,0.0,1.0,15
1,-1.533969,-1.517342,-1.340646,-1.274339,-1.218729,-1.237502,-1.270924,0.0,0.0,1.0,7
2,0.080351,0.162747,-0.065858,-0.258914,-0.447151,-0.319528,-0.130926,1.0,0.0,0.0,9
3,-0.726809,-0.433414,-0.349144,-0.621004,-0.648646,-0.590972,-0.578782,0.0,0.0,1.0,10
4,-1.713338,-1.625735,-1.623932,-1.320443,-1.267874,-1.326338,-1.393066,0.0,1.0,0.0,7
...,...,...,...,...,...,...,...,...,...,...,...
3776,0.394246,0.487926,0.784001,0.213376,0.110645,0.642863,0.186645,1.0,0.0,0.0,11
3777,0.618457,0.379533,-0.065858,0.391047,0.449746,0.401031,0.280287,0.0,0.0,1.0,10
3778,0.708142,0.758908,1.917145,0.863337,0.874851,1.121591,0.667072,0.0,0.0,1.0,9
3779,0.932353,0.867301,0.359071,0.680044,0.901881,0.860018,0.569358,1.0,0.0,0.0,10


In [109]:
X = abalone_df.iloc[:, :-1].values
y = abalone_df.iloc[:, -1].values

In [37]:
rs = 117

### Data splitting

In [110]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = rs)

### Model definition and training